In [1]:
import re
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [2]:
import os
with open(os.path.join("data_nutcha","data.txt"),"r") as f:
    data=[[j for j in striphtml(i).strip().replace("^^","^").replace("^^","^").replace("^^","^").split("^") if j!="" and j!=None]  for i in f.readlines()]

In [3]:
# with open(os.path.join(".","han_syllable.txt"),"r") as f:
#     han_syllable=[[j for j in i.strip().split("|") if j!="" and j!=None]  for i in f.readlines()]

In [4]:
from featurizer import Featurizer

to_feature=Featurizer()

In [5]:
def extract_features(l: list)->dict:
    c=0
    list_c = []
    list_y = []
    for i in l:
        for k,j in enumerate(i):
            if k==0:
                list_y.append("1")
            else:
                list_y.append("0")
            list_c.append(j)
    list_y.append("0")
    return {"X":to_feature.featurize(''.join(list_c))["X"],"y":list_y}

In [6]:
len(data)#,len(han_syllable)

(1877, 993)

In [7]:
temp=extract_features(data[0])

In [8]:
from sklearn.model_selection import train_test_split

In [10]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)
# han_syllable_train, han_syllable_test = train_test_split(han_syllable, test_size=0.2, random_state=42)

In [46]:
with open(os.path.join("dataset","han_solo_train.txt"),"r") as f:
    han_syllable_train=[[j for j in i.strip().split("|") if j!="" and j!=None]  for i in f.readlines()]
with open(os.path.join("dataset","han_solo_test.txt"),"r") as f:
    han_syllable_test=[[j for j in i.strip().split("|") if j!="" and j!=None]  for i in f.readlines()]

In [11]:
raw_test=data_test+data_test

In [12]:
train=[extract_features(s) for s in data_train+han_syllable_train]
test=[extract_features(s) for s in data_test+han_syllable_test]

In [13]:
X_train = [i["X"] for i in train]
y_train = [i["y"] for i in train]
X_test = [i["X"] for i in test]
y_test = [i["y"] for i in test]

In [14]:
X_train[0][-1]

['-1|0',
 '0|#',
 '-2|6',
 '1|#',
 '-3|น',
 '2|#',
 '-3|น6',
 '-2|60',
 '-1|0#',
 '0|##',
 '1|##']

In [15]:
len(X_train[1]),len(y_train[1])

(26, 26)

In [3]:
import pycrfsuite

In [17]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)
j=0

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

CPU times: user 3.58 s, sys: 68.3 ms, total: 3.65 s
Wall time: 3.65 s


In [18]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [19]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [20]:
%%time
trainer.train('han_solo.crfsuite')

CPU times: user 16.5 s, sys: 8.13 ms, total: 16.6 s
Wall time: 16.6 s


In [4]:
tagger = pycrfsuite.Tagger()
tagger.open('han_solo.crfsuite')

In [22]:
tagger.tag(X_test[0])

['1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0']

In [23]:
y_test[0]

['1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0']

In [24]:
from itertools import chain
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn

In [25]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [26]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

CPU times: user 594 ms, sys: 19.7 ms, total: 614 ms
Wall time: 613 ms


In [27]:
print(bio_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     61078
           1       1.00      0.99      0.99     29468

    accuracy                           1.00     90546
   macro avg       1.00      1.00      1.00     90546
weighted avg       1.00      1.00      1.00     90546



In [1]:
from ssg import syllable_tokenize

In [28]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
1      -> 0       1.995502
0      -> 0       1.588329
0      -> 1       1.058717
1      -> 1       -4.032953

Top unlikely transitions:
1      -> 0       1.995502
0      -> 0       1.588329
0      -> 1       1.058717
1      -> 1       -4.032953


In [29]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
14.462897 0      0|##
12.759949 0      1|์
11.508617 0      0|้
11.296093 0      0|า
11.002358 0      0|่
10.744058 0      2|์
10.551424 0      -1|เ
10.519173 0      0|ื
10.496157 1      -1|#
10.186005 0      0|ะ
10.083733 1      0|(
9.299560 1      0|ๆ
8.900142 0      0|ิ
8.700859 0      0|ั
8.052195 0      0|ี
7.986780 0      -1|ไ
7.637504 0      0|ู
7.576647 0      -1|แ
7.503324 0      0|ุ
7.211316 0      0|์

Top negative:
-2.832427 0      -2|ก็
-2.843872 0      -1|ำ
-2.992590 0      0|แ
-3.015528 1      -1|ตร
-3.092038 0      -1|ฯ
-3.118715 0      0|ไ
-3.125401 0      1|ู
-3.231541 0      0|เ
-3.317115 0      0|ใ
-3.436506 0      1|ำ
-3.592943 0      -1|)
-3.656251 0      -1|"
-3.761735 0      0|"
-3.920731 0      -1|(
-4.138270 0      0|ฯ
-4.150953 1      -1|ั
-4.354796 0      0|)
-4.902392 0      -1|ๆ
-6.360530 0      0| 
-6.405255 0      -1| 


In [30]:
def cut(text):
    x=to_feature.featurize(text)["X"]
    y_pred = tagger.tag(x)
    list_cut = []
    for i,(j,k) in enumerate(zip(list(text),y_pred)):
        if k=="1":
            list_cut.append(j)
        else:
            list_cut[-1]+=j
    return list_cut

In [31]:
cut("นายวรรณพงษ์ เป็นผู้พัฒนา")

['นาย', 'วรรณ', 'พงษ์', ' ', 'เป็น', 'ผู้', 'พัฒ', 'นา']

In [33]:
text = "✨น้องอร่อยมากเลยทุกคนนนนนน😭✨"#"ทดสอบการตัดพยางค์ภาษาไทย กันน:D"

In [34]:
cut(text)

['✨', 'น้อง', 'อร่อย', 'มาก', 'เลย', 'ทุก', 'คนนนนนน', '😭✨']

In [35]:
syllable_tokenize(text)

['✨', 'น้อง', 'อร่อย', 'มาก', 'เลย', 'ทุก', 'คน', 'นน', 'นน', 'น😭✨']

In [36]:
raw_test[0]

['1', 'กัน', 'ยา', 'ยน', '255000:21', 'น.']

In [37]:
cut(''.join(raw_test[0]))

['1', 'กัน', 'ยา', 'ยน', '255000:21', 'น.']

In [38]:
pre_model = ['|'.join(cut(''.join(i))) for i in raw_test]

In [39]:
y_raw_test = ['|'.join(i) for i in raw_test]

In [40]:
from pythainlp.benchmarks.word_tokenization import benchmark

In [41]:
benchmark(y_raw_test,pre_model)

,char_level:tp,char_level:fp,char_level:tn,char_level:fn,word_level:correctly_tokenised_words,word_level:total_words_in_sample,word_level:total_words_in_ref_sample,global:tokenisation_indicators,expected,actual
0,6,0,13,0,6,6.0,6.0,111111,1|กัน|ยา|ยน|255000:21|น.,1|กัน|ยา|ยน|255000:21|น.
1,85,2,177,1,84,87.0,86.0,1111111111111111111111111111111111111111100011...,อย่าง|ไร|ก็|ตาม|เจ้า|หน้า|ที่|ได้|แจ้ง|ข้อ|หา|...,อย่าง|ไร|ก็|ตาม|เจ้า|หน้า|ที่|ได้|แจ้ง|ข้อ|หา|...
2,22,0,49,0,22,22.0,22.0,1111111111111111111111,ลูก|โป่ง|คิด|ว่า|เป็น|ทาง|เลือก|สุด|ท้าย|ที่|ช...,ลูก|โป่ง|คิด|ว่า|เป็น|ทาง|เลือก|สุด|ท้าย|ที่|ช...
3,30,0,65,0,30,30.0,30.0,111111111111111111111111111111,พร้อม|กัน|นั้น|วัด|ทอง|ประ|ดิษฐ์|หมู่|2|ต.|บาง...,พร้อม|กัน|นั้น|วัด|ทอง|ประ|ดิษฐ์|หมู่|2|ต.|บาง...
4,10,0,17,0,10,10.0,10.0,1111111111,ใน|จำ|นวน|นี้|รัก|ษา|หาย|แล้ว|10|ราย,ใน|จำ|นวน|นี้|รัก|ษา|หาย|แล้ว|10|ราย
...,...,...,...,...,...,...,...,...,...,...
747,4,0,12,0,4,4.0,4.0,1111,แบงก์|ชาติ|ถูก|ต้อง,แบงก์|ชาติ|ถูก|ต้อง
748,19,0,44,0,19,19.0,19.0,1111111111111111111,อย่าง|ไร|ก็|ตาม|ยัง|ไม่|ได้|ประ|กาศ|พื้น|ที่|ส...,อย่าง|ไร|ก็|ตาม|ยัง|ไม่|ได้|ประ|กาศ|พื้น|ที่|ส...
749,37,0,88,0,37,37.0,37.0,1111111111111111111111111111111111111,ขณะ|ที่|ก่อน|หน้า|นี้|ญี่|ปุ่น|สั่ง|ห้าม|นำ|เข...,ขณะ|ที่|ก่อน|หน้า|นี้|ญี่|ปุ่น|สั่ง|ห้าม|นำ|เข...
750,6,0,13,0,6,6.0,6.0,111111,1|กัน|ยา|ยน|255015:36|น.,1|กัน|ยา|ยน|255015:36|น.
